### 세팅

In [ ]:
# ! pip install konlpy wordcloud matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 kB 28.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get update -qq
!apt-get install -y fonts-nanum
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (6,189 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
import matplotlib as mpl

# 시스템 폰트 디렉토리 내 나눔고딕 경로
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"

# 폰트 수동 등록
fm.fontManager.addfont(font_path)
nanum_font = fm.FontProperties(fname=font_path).get_name()

# matplotlib에 폰트 등록
plt.rc('font', family=nanum_font)
plt.rcParams['axes.unicode_minus'] = False

# 확인
print(f"✅ 적용된 폰트 이름: {nanum_font}")

✅ 적용된 폰트 이름: NanumGothic


In [ ]:
# 1. 라이브러리 불러오기
import os
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
base_path = "/content"

### 클러스터별 워드 클라우드 생성

불용어 설정

In [ ]:
custom_stopwords = set([
    '일본', '日', '후쿠시마', '오염수', '방류', '원전', '정부',
    '관련', '위해', '대한', '있다', '없다', '등', '또', '중', '그리고', '포토', '가', '최근', '고 했다', '수 있는', '이라고 말했다', '같은', '고 지적했다', '고', '등의', '지금', '이라고', '사설',
    '의','에','은','는','이','가',
    '로','으로','와','과','및','에서','에도',
    '하고','도','만','중','대한','대해',
    '기자','보도',
    '공개','결정','발표','추진'
    '회의','논의','발언','답변','질의',
    '이번','최근','만에','첫','두번째',
    '방출', '尹', '與', '野', '없어', '왜', '윤 대통령', '이재명', '것', '민주', '민주당'
    ])


클러스터별 워드 클라우드 생성

In [ ]:
group_dict = {
    "조중동": ["중앙일보_최종.xlsx", "동아일보_최종.xlsx", "조선일보_최종.xlsx"],
    "한한경": ["한겨레_최종.xlsx", "한국일보_최종.xlsx", '경향신문_최종.xlsx'],
    "문국세": ["문화일보_최종.xlsx", "국민일보_최종.xlsx", "세계일보_최종.xlsx"],
    "내아서": ["내일신문_최종.xlsx", "아시아투데이_최종.xlsx", "서울신문_최종.xlsx"]
}

In [ ]:
import random
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import os
import pandas as pd

# ✅ 그룹별 색상 매핑 함수 정의
def red_color_func(*args, **kwargs):
    return f"rgb({random.randint(180, 255)}, {random.randint(0, 80)}, {random.randint(0, 80)})"

def blue_color_func(*args, **kwargs):
    return f"rgb({random.randint(0, 80)}, {random.randint(100, 180)}, {random.randint(200, 255)})"

def pink_color_func(*args, **kwargs):
    return f"rgb({random.randint(230, 255)}, {random.randint(150, 200)}, {random.randint(180, 220)})"

def green_color_func(*args, **kwargs):
    return f"rgb({random.randint(0, 80)}, {random.randint(150, 255)}, {random.randint(0, 80)})"

# ✅ 그룹별 색상 함수 지정
color_func_map = {
    "조중동": red_color_func,
    "한한경": blue_color_func,
    "문국세": pink_color_func,
    "내아서": green_color_func
}

# 저장 경로 생성
save_path = os.path.join(base_path, "group_wordclouds_colored")
os.makedirs(save_path, exist_ok=True)

# 그룹별 워드클라우드 생성
for group_name, files in group_dict.items():
    all_titles = []

    for file in files:
        file_path = os.path.join(base_path, file)

        try:
            df = pd.read_excel(file_path)
        except Exception as e:
            print(f"[{file}] 읽기 오류:", e)
            continue

        # 제목 컬럼 자동 탐지
        title_col = [col for col in df.columns if '제목' in col or 'title' in col.lower()]
        if not title_col:
            print(f"[{file}] 제목 컬럼 없음")
            continue

        titles = df[title_col[0]].dropna().astype(str).tolist()
        all_titles.extend(titles)

    if not all_titles:
        print(f"[{group_name}] 제목 없음")
        continue

    # 하나의 문자열로 합치기
    text = " ".join(all_titles)

    # 워드클라우드 생성
    wc = WordCloud(
        font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
        background_color='white',
        width=800,
        height=400,
        stopwords=custom_stopwords
    ).generate(text)

    # 색상 함수 선택
    color_func = color_func_map.get(group_name, None)

    # 시각화 및 저장
    plt.figure(figsize=(10, 5))
    plt.imshow(wc.recolor(color_func=color_func), interpolation='bilinear')
    plt.axis('off')
    plt.title(f"{group_name} 워드클라우드")
    plt.tight_layout()

    save_file = os.path.join(save_path, f"{group_name}_wordcloud.png")
    plt.savefig(save_file)
    plt.close()

    print(f"✔️ {group_name} 워드클라우드 저장됨: {save_file}")


✔️ 조중동 워드클라우드 저장됨: /content/group_wordclouds_colored/조중동_wordcloud.png
✔️ 한한경 워드클라우드 저장됨: /content/group_wordclouds_colored/한한경_wordcloud.png
✔️ 문국세 워드클라우드 저장됨: /content/group_wordclouds_colored/문국세_wordcloud.png
✔️ 내아서 워드클라우드 저장됨: /content/group_wordclouds_colored/내아서_wordcloud.png
